# Akbank Derin Öğrenme Bootcamp Projesi: Maymun Türleri Sınıflandırması

Bu proje, Akbank Derin Öğrenme Bootcamp'i kapsamında gerçekleştirilmiştir. Projenin temel amacı, Evrişimli Sinir Ağları (CNN) kullanarak bir görüntü sınıflandırma problemini baştan sona uygulamaktır. Veri seti olarak Kaggle'da bulunan ve 10 farklı maymun türüne ait görseller içeren "10 Monkey Species" veri seti seçilmiştir. Bu notebook, veri ön işleme, model geliştirme, eğitim, değerlendirme ve optimizasyon adımlarını içermektedir.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings("ignore")

## 1. Veri Setinin İncelenmesi ve Keşfi (Exploratory Data Analysis - EDA)
Bu bölümde ilk olarak veri setimizin yapısı incelenmektedir. `matplotlib` ve `seaborn` kütüphaneleri kullanılarak her bir sınıftaki (maymun türü) görüntü sayısı görselleştirilmiş ve sınıflar arasında dengeli bir dağılım olduğu gözlemlenmiştir. Ayrıca, her sınıftan rastgele birer örnek görseli ekrana çizdirilerek veri seti hakkında genel bir fikir edinilmiştir.

In [ ]:
base_dir = '../input/10-monkey-species'
train_dir = os.path.join(base_dir, 'training/training')
validation_dir = os.path.join(base_dir, 'validation/validation')

class_names = os.listdir(train_dir)
class_names.sort()
print("Toplam sınıf (maymun türü) sayısı:", len(class_names))
print("Sınıf isimleri:", class_names)

train_dir = '/kaggle/input/10-monkey-species/training/training'
class_names = os.listdir(train_dir)
image_counts = {}
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    image_counts[class_name] = len(os.listdir(class_path))

plt.figure(figsize=(12, 6))
sns.barplot(x=list(image_counts.keys()), y=list(image_counts.values()))
plt.title('Her Sınıftaki Eğitim Görüntüsü Sayısı')
plt.xlabel('Maymun Türü')
plt.ylabel('Görüntü Sayısı')
plt.xticks(rotation=45)
plt.show()
warnings.filterwarnings("ignore")

import random
from tensorflow.keras.preprocessing.image import load_img

plt.figure(figsize=(15, 10))

for i, class_name in enumerate(class_names):
    # Her sınıf klasöründen rastgele bir resim seç
    class_path = os.path.join(train_dir, class_name)
    random_image = random.choice(os.listdir(class_path))
    image_path = os.path.join(class_path, random_image)

    img = load_img(image_path)
    ax = plt.subplot(2, 5, i + 1)
    plt.imshow(img)
    plt.title(f"{i}: {class_name}") # .split() kısmını kaldırdık
    plt.axis("off")

plt.show()

## 2. Veri Ön İşleme ve Veri Çoğaltma (Data Preprocessing & Augmentation)
Bu aşamada, görseller modelin işleyebileceği formata getirilmiştir. `ImageDataGenerator` kullanılarak veri ön işleme ve çoğaltma (Data Augmentation) adımları gerçekleştirilmiştir.

* **Veri Çoğaltma:** Modelin genelleme yeteneğini artırmak ve ezberlemesini (overfitting) önlemek amacıyla eğitim setindeki görüntülere rastgele döndürme (rotation), yakınlaştırma (zoom) ve yatay çevirme (flip) gibi çeşitli dönüşümler uygulanmıştır.
* **Ön İşleme:** Kullandığımız `EfficientNetB0` modeli kendi içinde bir ön işleme (rescale) katmanına sahip olduğu için, veri jeneratöründe ayrıca bir piksel normalizasyonu (`rescale=1./255`) yapılmamıştır. Veriler modele orijinal piksel değerleri (0-255) ile beslenmiştir.

In [ ]:
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Eğitim verisi hazırlanıyor...")
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

print("Doğrulama verisi hazırlanıyor...")
validation_datagen = ImageDataGenerator() 

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
print("Veri jeneratörleri başarıyla oluşturuldu.")

## 3. Model Mimarisi: Transfer Learning ile EfficientNetB0
Projenin bu aşamasında, yüksek başarı oranına ulaşmak için **Transfer Learning (Transfer Öğrenme)** tekniği kullanılmıştır. Sıfırdan bir CNN modeli eğitmek yerine, Google tarafından geliştirilen ve ImageNet veri setinde eğitilmiş olan `EfficientNetB0` modeli temel (base) olarak alınmıştır.

Modelin mimarisi şu şekildedir:
1.  **Temel Model:** `EfficientNetB0` modelinin en üstündeki sınıflandırma katmanı olmadan (`include_top=False`) yüklenmiştir.
2.  **Dondurma (Freezing):** `EfficientNetB0`'ın önceden öğrenilmiş olan ağırlıklarının bozulmaması için temel modelin tüm katmanları dondurulmuştur.
3.  **Özel Sınıflandırıcı Katmanları:** Dondurulmuş temel modelin çıktısına, kendi problemimize özel bir sınıflandırıcı eklenmiştir. Bu sınıflandırıcıda `GlobalAveragePooling2D`, `Dense` ve `Dropout` katmanları kullanılmıştır. Çıkış katmanında 10 sınıfımız için `softmax` aktivasyon fonksiyonu tercih edilmiştir.

In [ ]:
# --- TRANSFER LEARNING MODELİ KURULUMU, EĞİTİMİ VE DEĞERLENDİRMESİ ---

# Gerekli kütüphaneler
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Adım 1: Önceden eğitilmiş EfficientNetB0 modelinin yüklenmesi
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Adım 2: Temel modelin katmanlarının dondurulmesi
base_model.trainable = False

# Adım 3: Kendi sınıflandırıcımızı temel modelin üzerine eklenmesi
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)

# Adım 4: Yeni model oluşturulur ve derlenir
model = Model(inputs=base_model.input, outputs=outputs)

# Transfer learning için düşük bir öğrenme oranı kullanıyoruz.
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("--- TRANSFER LEARNING MODELİ BAŞARIYLA OLUŞTURULDU ---")
model.summary()

## 4. Modelin Eğitilmesi
Model, `Adam` optimize edici ve `categorical_crossentropy` kayıp fonksiyonu ile derlenmiştir. Transfer Learning tekniğinde, önceden eğitilmiş ağırlıklara hassas bir şekilde adapte olabilmek için düşük bir öğrenme oranı (`learning_rate=0.0001`) seçilmiştir.

In [ ]:
# Adım 5: Yeni modelin eğitilmesi
EPOCHS = 30
print(f"\n--- TRANSFER LEARNING EĞİTİMİ BAŞLATILIYOR ({EPOCHS} EPOCH) ---")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

## 5. Eğitim Sürecinin Değerlendirilmesi
Model 30 epoch boyunca eğitilmiştir. Aşağıdaki grafikler, eğitim süresince modelin doğruluk ve kayıp metriklerinin değişimini göstermektedir. Grafiklerin incelenmesi, modelin başarılı bir şekilde öğrenip öğrenmediğini ve ezberleme (overfitting) problemi yaşayıp yaşamadığını anlamak için kritik öneme sahiptir.

In [ ]:
# Adım 6: Eğitim sonuçlarının görselleştirilmesi
print("\n--- EĞİTİM GRAFİKLERİ OLUŞTURULUYOR ---")
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Eğitim Doğruluğu')
plt.plot(range(EPOCHS), val_acc, label='Doğrulama Doğruluğu')
plt.legend(loc='lower right')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Eğitim Kaybı')
plt.plot(range(EPOCHS), val_loss, label='Doğrulama Kaybı')
plt.legend(loc='upper right')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.show()

## 6. Nihai Model Performansı
Eğitim sürecinin görsel analizinden sonra, modelin nihai performansı doğrulama seti üzerinde sayısal metriklerle değerlendirilmiştir. `Classification Report` her bir sınıf için precision, recall ve f1-score değerlerini sunarken, `Confusion Matrix` modelin hangi sınıfları birbiriyle karıştırdığını görsel olarak göstermektedir.

In [ ]:
# Adım 7: Nihai Model Değerlendirmesi
print("\n--- NİHAİ MODEL DEĞERLENDİRMESİ ---")
validation_generator.reset()
Y_pred = model.predict(validation_generator, steps=validation_generator.samples // BATCH_SIZE + 1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes

print("\nClassification Report")
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Confusion Matrix')
plt.show()

## 7. Sonuç ve Proje Özeti
Bu projenin sonucunda, Transfer Learning ve `EfficientNetB0` mimarisi kullanılarak 10 farklı maymun türünü sınıflandıran yüksek başarılı bir model geliştirilmiştir. Veri çoğaltma ve düşük öğrenme oranı gibi optimizasyon teknikleri sayesinde modelin ezberleme yapması engellenmiş ve doğrulama seti üzerinde **~%97** gibi oldukça yüksek bir doğruluk oranına ulaşılmıştır. Bu sonuç, projenin başlangıcında belirlenen hedeflere başarıyla ulaşıldığını göstermektedir.